# Clone repo and install dependencies

In [1]:
!git clone https://github.com/cloneofsimo/lora.git && sed -i 's/functools.cache/functools.lru_cache(maxsize=None)/g' /content/lora/lora_diffusion/xformers_utils.py && pip install /content/lora
!pip install jupyter
!pip install cuda-python
!pip install accelerate==0.17.0 bitsandbytes
!pip install cuda-python
!pip install -r ./lora/requirements.txt
!nvcc --version
# !pip uninstall -y accelerate
# !pip install accelerate==0.17.0

fatal: destination path 'lora' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 49.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 4.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 16.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [18]:
!ls /datasets/jpg-training

1.jpeg	 12.jpeg  15.jpeg  18.jpeg  20.jpeg  5.jpeg  8.jpeg
10.jpeg  13.jpeg  16.jpeg  19.jpeg  3.jpeg   6.jpeg  9.jpeg
11.jpeg  14.jpeg  17.jpeg  2.jpeg   4.jpeg   7.jpeg


# Set directories

In [5]:
import os
import shutil
#from google.colab import files
from tqdm import tqdm

PRETRAINED_MODEL="runwayml/stable-diffusion-v1-5" #@param{type: 'string'}
PROMPT="erosion" #@param{type: 'string'}

OUTPUT_DIR="/notebooks/lora/outputs/lorajpg200" #@param{type: 'string'}
IMAGES_FOLDER_OPTIONAL="/datasets/jpg-training-200" #@param{type: 'string'}

RESOLUTION="512" #@param ["512", "576", "640", "704", "768", "832", "896", "960", "1024"]
RESOLUTION=int(RESOLUTION)

if PRETRAINED_MODEL == "":
  print('[1;31mYou should define the pretrained model.')

else:
  if IMAGES_FOLDER_OPTIONAL=="":
    INSTANCE_DIR = "/content/data_example"
    if not os.path.exists(str(INSTANCE_DIR)):
      %mkdir -p "$INSTANCE_DIR"
    uploaded = files.upload()
    for filename in tqdm(uploaded.keys(), bar_format='  |{bar:15}| {n_fmt}/{total_fmt} Uploaded'):
        shutil.move(filename, INSTANCE_DIR)
  else:
    INSTANCE_DIR = IMAGES_FOLDER_OPTIONAL
  
  if OUTPUT_DIR == "":
    OUTPUT_DIR = "/content/output"
  if not os.path.exists(str(OUTPUT_DIR)):
    %mkdir -p "$OUTPUT_DIR"

# Run training

In [ ]:
STEPS = 10000 #@param {type:"slider", min:0, max:10000, step:10}
BATCH_SIZE = 1 #@param {type:"slider", min:0, max:128, step:1}
FP_16 = True #@param {type:"boolean"}

#@markdown ----
#@markdown UNET PARAMS
LEARNING_RATE = 1e-4 #@param {type:"number"}

#@markdown ----
TRAIN_TEXT_ENCODER = True #@param {type:"boolean"}
#@markdown TEXT ENCODER PARAMS
LEARNING_RATE_TEXT_ENCODER = 5e-5 #@param {type:"number"}

NEW_LEARNING_RATE = LEARNING_RATE / BATCH_SIZE
NEW_LEARNING_RATE_TEXT_ENCODER = LEARNING_RATE_TEXT_ENCODER / BATCH_SIZE

if FP_16:
  fp_16_arg = "fp16"
else:
  fp_16_arg = "no"

# train_command = f"""accelerate launch lora/training_scripts/train_lora_dreambooth.py \
#              --pretrained_model_name_or_path="{PRETRAINED_MODEL}" \
#              --instance_data_dir="{INSTANCE_DIR}" \
#              --output_dir="{OUTPUT_DIR}" \
#              --instance_prompt="{PROMPT}" \
#              --resolution=512 \
#              --use_8bit_adam \
#              --mixed_precision="{fp_16_arg}" \
#              --train_batch_size={BATCH_SIZE} \
#              --gradient_accumulation_steps=1 \
#              --learning_rate={NEW_LEARNING_RATE} \
#              --lr_scheduler="constant" \
#              --lr_warmup_steps=0 \
#              --max_train_steps={STEPS} \
#              --train_text_encoder \
#              --lora_rank=16 \
#              --learning_rate_text={NEW_LEARNING_RATE_TEXT_ENCODER}"""

# print(train_command)
# f = open("./train.sh", "w")
# f.write(train_command)
# f.close()
# !chmod +x ./train.sh
# !./train.sh

if TRAIN_TEXT_ENCODER:
  command = (f'python3 lora/training_scripts/train_lora_dreambooth.py '
             f'--pretrained_model_name_or_path="{PRETRAINED_MODEL}" '
             f'--instance_data_dir="{INSTANCE_DIR}" '
             f'--output_dir="{OUTPUT_DIR}" '
             f'--instance_prompt="{PROMPT}" '
             f'--resolution=512 '
             f'--mixed_precision="{fp_16_arg}" '
             f'--train_batch_size={BATCH_SIZE} '
             f'--gradient_accumulation_steps=1 '
             f'--learning_rate={NEW_LEARNING_RATE} '
             f'--lr_scheduler="constant" '
             f'--lr_warmup_steps=0 '
             f'--max_train_steps={STEPS} '
             f'--train_text_encoder '
             f'--lora_rank=16 '
             f'--learning_rate_text={NEW_LEARNING_RATE_TEXT_ENCODER}')
else:
  command = (f'accelerate launch lora/training_scripts/train_lora_dreambooth.py '
             f'--pretrained_model_name_or_path="{PRETRAINED_MODEL}" '
             f'--instance_data_dir="{INSTANCE_DIR}" '
             f'--output_dir="{OUTPUT_DIR}" '
             f'--instance_prompt="{PROMPT}" '
             f'--resolution=512 '
             f'--use_8bit_adam '
             f'--mixed_precision="{fp_16_arg}" '
             f'--train_batch_size=1 '
             f'--gradient_accumulation_steps={BATCH_SIZE} '
             f'--learning_rate={NEW_LEARNING_RATE} '
             f'--lr_scheduler="constant" '
             f'--lr_warmup_steps=0 '
             f'--lora_rank=16 '
             f'--max_train_steps={STEPS} '
             f'--learning_rate_text={NEW_LEARNING_RATE_TEXT_ENCODER}')
!rm -rf $INSTANCE_DIR/.ipynb_checkpoints
!{command}

/usr/local/lib/python3.9/dist-packages/accelerate/accelerator.py:243: FutureWarning: `logging_dir` is deprecated and will be removed in version 0.18.0 of 🤗 Accelerate. Use `project_dir` instead.
  warnings.warn(

In [6]:
import sys
print(sys.executable)
print(sys.path)
#sys.path.append('/usr/local/bin/python3')

/usr/local/bin/python3
['/notebooks', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/bin/python3']


Zip models and logs

In [3]:
!cd /notebooks
!zip -r jpg20.zip lora/outputs/lorajpg

  adding: lora/outputs/lorajpg/ (stored 0%)
  adding: lora/outputs/lorajpg/lora_weight_e224_s4500.pt (deflated 7%)
  adding: lora/outputs/lorajpg/lora_weight_e249_s5000.text_encoder.pt (deflated 6%)
  adding: lora/outputs/lorajpg/lora_weight_e399_s8000.text_encoder.pt (deflated 6%)
  adding: lora/outputs/lorajpg/lora_weight_e449_s9000.pt (deflated 7%)
  adding: lora/outputs/lorajpg/lora_weight_e99_s2000.text_encoder.pt (deflated 6%)
  adding: lora/outputs/lorajpg/lora_weight_e249_s5000.pt (deflated 7%)
  adding: lora/outputs/lorajpg/lora_weight_e149_s3000.pt (deflated 7%)
  adding: lora/outputs/lorajpg/lora_weight.pt (deflated 7%)
  adding: lora/outputs/lorajpg/lora_weight_e74_s1500.pt (deflated 7%)
  adding: lora/outputs/lorajpg/lora_weight_e399_s8000.pt (deflated 7%)
  adding: lora/outputs/lorajpg/lora_weight_e424_s8500.text_encoder.pt (deflated 6%)
  adding: lora/outputs/lorajpg/lora_weight_e99_s2000.pt (deflated 7%)
  adding: lora/outputs/lorajpg/lora_weight_e324_s6500.pt (deflated